In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
!{sys.executable} -m pip install mlxtend

In [4]:
import mlxtend
print(mlxtend.__version__)


0.23.4


In [5]:
import sqlite3

In [6]:
import pandas as pd

In [19]:
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth



# Northwind Market Basket Analysis

In [8]:
conn = sqlite3.connect(r"D:\northwind_project\northwind-SQLite3-main\dist\northwind.db")

In [9]:
orders = pd.read_sql('SELECT * FROM Orders;', conn)
order_details = pd.read_sql('SELECT * FROM [Order Details];', conn)
products = pd.read_sql('SELECT * FROM Products;', conn)
categories = pd.read_sql('SELECT * FROM Categories;', conn)

In [10]:
conn.close()

In [11]:
df = order_details.merge(orders, on='OrderID', how='left')
df = df.merge(products, on='ProductID', how='left')
df = df.merge(categories, on='CategoryID', how='left')

In [12]:
df.dropna(subset=['ProductName'], inplace=True)
df.drop_duplicates(inplace=True)

In [13]:
basket_size = df.groupby('OrderID')['ProductID'].count()
print("Basket size description:\n", basket_size.describe())

Basket size description:
 count    16282.000000
mean        37.420649
std         23.108610
min          1.000000
25%         17.000000
50%         37.000000
75%         58.000000
max         77.000000
Name: ProductID, dtype: float64


In [14]:
top_products = df['ProductName'].value_counts().head(10)
print("Top products:\n", top_products)

Top products:
 ProductName
Louisiana Hot Spiced Okra     8040
Teatime Chocolate Biscuits    8024
Outback Lager                 8020
Gumbär Gummibärchen           7999
Sir Rodney's Marmalade        7999
Gudbrandsdalsost              7991
Raclette Courdavault          7982
Ravioli Angelo                7969
Konbu                         7968
Gorgonzola Telino             7964
Name: count, dtype: int64


In [15]:
top_categories = df['CategoryName'].value_counts()
print("Top categories:\n", top_categories)

Top categories:
 CategoryName
Confections       102784
Beverages          95083
Condiments         94988
Seafood            94912
Dairy Products     79233
Grains/Cereals     55400
Meat/Poultry       47335
Produce            39548
Name: count, dtype: int64


In [16]:
basket = df.groupby(['OrderID','ProductName'])['Quantity'].sum().unstack().fillna(0)
basket_binary = basket.applymap(lambda x: 1 if x>0 else 0)

In [ ]:
# frequent_itemsets = apriori(basket_binary, min_support=0.02, use_colnames=True)

basket_binary = basket_binary.astype(bool)

frequent_itemsets = fpgrowth(
    basket_binary,
    min_support=0.05, 
    use_colnames=True
)


KeyboardInterrupt: 

In [ ]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)
rules_filtered = rules[(rules['confidence']>0.4) & (rules['lift']>1.2)]
print("Filtered rules:\n", rules_filtered.head())

In [ ]:
basket_cat = df.groupby(['OrderID','CategoryName'])['Quantity'].sum().unstack().fillna(0)
basket_cat_binary = basket_cat.applymap(lambda x: 1 if x>0 else 0)


In [ ]:
frequent_cat = apriori(basket_cat_binary, min_support=0.05, use_colnames=True)
rules_cat = association_rules(frequent_cat, metric='lift', min_threshold=1.0)
rules_cat.sort_values('lift', ascending=False).head()

In [ ]:
rules_filtered.to_csv('association_rules_products.csv', index=False)
rules_cat.to_csv('association_rules_categories.csv', index=False)
